In [ ]:
# default_exp preprocessing

# Preprocessing

> Este modulo es el encargado de la carga, preprocesamiento y preparacion de los datos.

In [ ]:
#hide
from nbdev.showdoc import *
import numpy as np
import pandas as pd
import re
import unittest
from obspy import read
from pathlib import Path

## Clase TimeSerie

In [ ]:
#export
class TimeSerie:
    "Estructura de datos que almacena las mediciones en una estacion."
    
    @staticmethod
    def __process_time(time):
        match = re.match(r"(?P<val>\d+)(?P<smh>[smh]*)$", time, re.I)
        assert match, "Formato incorrecto de tamaño de ventana o paso."
        
        result = int(match.group("val"))
        if match.group("smh").lower() == "m":
            result *= 60
        elif match.group("smh").lower() == "h":
            result *= 60*60
        return result
        
        
    def __init__(self, window_size:str="2m", step:str="30s", freq:int=50):
        "El formato de `window_size` y `step` es \d+[smh]"
        "`window_size` representa el tamaño de tiempo en que sera cortada la serie de tiempo"
        "`step` representa el tamaño de paso que se utiliza al cortar la serie de tiempo"
        "`freq` es la frecuencia en Hz que tienen las mediciones"
        
        self.window_size = self.__process_time(window_size)
        self.step = self.__process_time(step)
        
        assert type(freq) is int, "La frecuencia debe ser un entero."
        assert freq > 0 and freq <= 500, "La frecuencia debe definirse entre 1 y 500 Hz"
        self.freq = freq
        
    @staticmethod
    def load_data_one_file(file_path, station="B2DF", channel="001"):
        """
        Busca en el camino del archivo pasado por parametros y
        retorna un diccionario con la fecha de inicio y termino, 
        ademas de los datos correspondientes a la estacion y el canal pasados por parametros.
        """
        st = read(str(file_path))
        for tr in st:
            if tr.stats.station == station and tr.stats.channel == channel:
                return {"start": tr.stats.starttime, "end": tr.stats.endtime, "data": tr.data}
    
    @staticmethod
    def load_data_list(list_file_path, station="B2DF", channel="001"):
        """
        Busca en cada camino de archivo de la lista pasado por parametros y
        retorna una lista de diccionarios con las fecha de inicio y termino, 
        ademas de los datos correspondientes a la estacion y el canal pasados por parametros.
        """
        data = [ 
                 TimeSerie.load_data_one_file(file_path, station, channel) \
                 for file_path in list_file_path \
                 if not file_path.is_dir()
               ]
        return data
    
    @staticmethod
    def load_data_folder(folder_path, station="B2DF", channel="001"):
        """
        Busca en cada archivo miniseed del directorio pasado por parametros y
        retorna una lista de diccionarios con las fecha de inicio y termino, 
        ademas de los datos correspondientes a la estacion y el canal pasados por parametros.
        """
        data = TimeSerie.load_data_list(folder_path.glob("**/*"), station, channel)
        return data
    
    def __repr__(self):
        return f"window_size {self.window_size}\nstep {self.step}\nfreq {self.freq}"
    
    def __str__(self):
        return f"window_size {self.window_size}\nstep {self.step}\nfreq {self.freq}"

In [ ]:
show_doc(TimeSerie.load_data_one_file)

<h4 id="TimeSerie.load_data_one_file" class="doc_header"><code>TimeSerie.load_data_one_file</code><a href="__main__.py#L31" class="source_link" style="float:right">[source]</a></h4>

> <code>TimeSerie.load_data_one_file</code>(**`file_path`**, **`station`**=*`'B2DF'`*, **`channel`**=*`'001'`*)

Busca en el camino del archivo pasado por parametros y
retorna un diccionario con la fecha de inicio y termino, 
ademas de los datos correspondientes a la estacion y el canal pasados por parametros.

In [ ]:
show_doc(TimeSerie.load_data_folder)

<h4 id="TimeSerie.load_data_folder" class="doc_header"><code>TimeSerie.load_data_folder</code><a href="__main__.py#L53" class="source_link" style="float:right">[source]</a></h4>

> <code>TimeSerie.load_data_folder</code>(**`folder_path`**, **`station`**=*`'B2DF'`*, **`channel`**=*`'001'`*)

Busca en cada archivo miniseed del directorio pasado por parametros y
retorna una lista de diccionarios con las fecha de inicio y termino, 
ademas de los datos correspondientes a la estacion y el canal pasados por parametros.

### Ejemplos:

Serie de tiempo con ventana de 2 minutos y paso de 30 segundos, el muestreo fue hecho a 50 Hz.

In [ ]:
TimeSerie("2m","30s",50)

In [ ]:
TimeSerie("4s","2s",40)

In [ ]:
TimeSerie("1h","30m",10)

In [ ]:
TimeSerie("10","5",15)

In [ ]:
TimeSerie("2m","30",30)

### Tests

In [ ]:

#Test definition

class TestTimeSerie(unittest.TestCase):
    
    def __check_params(self, ts, window_size, step, freq):
        self.assertEqual(ts.window_size, window_size)
        self.assertEqual(ts.step, step)
        self.assertEqual(ts.freq, freq)
    
    def test_params(self):
        ts = TimeSerie("2m","30s",50)
        self.__check_params(ts, 120, 30, 50)
        
        ts = TimeSerie("4s","2s",40)
        self.__check_params(ts, 4, 2, 40)
        
        ts = TimeSerie("1h","30m",10)
        self.__check_params(ts, 60*60, 30*60, 10)
        
        ts = TimeSerie("10","5",15)
        self.__check_params(ts, 10, 5, 15)
        
        ts = TimeSerie("2m","30",30)
        self.__check_params(ts, 2*60, 30, 30)
    
    def test_file_load(self):
        res = TimeSerie.load_data_one_file(Path("./data/B2DF/raw/2018-10-30-2100-00M.B2DF__001_00_01"))
        res["data"] = res["data"][:10]
        pd.DataFrame(res).to_csv(Path("./data/B2DF/test/csv")/'test_load_file1.csv', index=False)
        
        df1 = pd.read_csv(Path("./data/B2DF/test/csv/test_load_file1.csv"))
        df2 = pd.read_csv(Path("./data/B2DF/test/csv/test_load_file.csv"))
        pd.util.testing.assert_frame_equal(df1, df2)
    
    def test_list_load(self):
        files = [
                    Path("./data/B2DF/raw/2018-10-30-2100-00M.B2DF__001_00_01"),
                    Path("./data/B2DF/raw/2018-10-30-2200-00M.B2DF__001_00_01"),
                    Path("./data/B2DF/raw/2018-10-30-2300-00M.B2DF__001_00_01"),
                ]
        
        pd.DataFrame(TimeSerie.load_data_list(files)) \
            .to_csv(Path("./data/B2DF/test/csv")/'test_load_list1.csv', index=False)
        
        df1 = pd.read_csv(Path("./data/B2DF/test/csv/test_load_list1.csv"))
        df2 = pd.read_csv(Path("./data/B2DF/test/csv/test_load_list.csv"))
        pd.util.testing.assert_frame_equal(df1, df2)
    
    def test_folder_load(self):
        pd.DataFrame(TimeSerie.load_data_folder(Path("./data/B2DF/test/raw"))) \
            .to_csv(Path("./data/B2DF/test/csv")/'test_load_folder1.csv', index=False)
        
        df1 = pd.read_csv(Path("./data/B2DF/test/csv/test_load_folder1.csv"))
        df2 = pd.read_csv(Path("./data/B2DF/test/csv/test_load_folder.csv"))
        pd.util.testing.assert_frame_equal(df1, df2)

In [ ]:
#hide

#Test run
unittest.main(argv=[''], verbosity=2, exit=False)

test_file_load (__main__.TestTimeSerie) ... ok
test_folder_load (__main__.TestTimeSerie) ... ok
test_list_load (__main__.TestTimeSerie) ... ok
test_params (__main__.TestTimeSerie) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.192s

OK


#hide
## For testing only 

In [ ]:
# hide
def load_first_file(data_path): 
    for idx, file_path in enumerate(data_path.glob("**/*")):
        if not file_path.is_dir():
            return file_path

In [ ]:
#hide
